In [5]:
#pip install nltk

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: nltk in d:\au\sem2\dl lab\dl_lab\lib\site-packages (3.7)



You should consider upgrading via the 'D:\AU\sem2\DL lab\dl_lab\Scripts\python.exe -m pip install --upgrade pip' command.


In [7]:
# import nltk
# import ssl

# try:
#     _create_unverified_https_context = ssl._create_unverified_context
# except AttributeError:
#     pass
# else:
#     ssl._create_default_https_context = _create_unverified_https_context

# nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mortal\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [2]:
#import nltk
#nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


False

In [17]:
import numpy as np
import pandas as pd 
import time 
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
english_stop_words = stopwords.words('english')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
import seaborn as sns
sns.set(style ='whitegrid')
pd.set_option('display.max_columns',None)

In [19]:
headers=['Tweet_ID','Entity','Sentiment','Tweet_content']
train_df=pd.read_csv('twitter_training.csv', names=headers)
valid_df=pd.read_csv('twitter_validation.csv', names=headers)

In [20]:
train_df.head()

,Tweet_ID,Entity,Sentiment,Tweet_content
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [21]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74682 entries, 0 to 74681
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Tweet_ID       74682 non-null  int64 
 1   Entity         74682 non-null  object
 2   Sentiment      74682 non-null  object
 3   Tweet_content  73996 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.3+ MB


In [22]:
train_df['Sentiment'].value_counts()

Negative      22542
Positive      20832
Neutral       18318
Irrelevant    12990
Name: Sentiment, dtype: int64

In [23]:
train_df['Tweet_content'].isnull().sum()

686

In [24]:
train_df.dropna(axis=0, inplace=True)

In [25]:
tweet_train  = train_df["Tweet_content"]
tweet_valid=valid_df["Tweet_content"]
target=train_df['Sentiment']

In [26]:
REPLACE_WITH_SPACE = re.compile("(@)")
SPACE = " "
def preprocess_reviews(reviews):  
    reviews = [REPLACE_WITH_SPACE.sub(SPACE, line.lower()) for line in reviews]
    return reviews
reviews_train_clean = preprocess_reviews(tweet_train)
reviews_valid_clean = preprocess_reviews(tweet_valid)

In [27]:
def remove_stop_words(corpus):
    removed_stop_words = []
    for review in corpus:
        removed_stop_words.append(
            ' '.join([word for word in review.split()  if word not in english_stop_words]))
    return removed_stop_words
no_stop_words_train = remove_stop_words(reviews_train_clean)
no_stop_words_valid = remove_stop_words(reviews_valid_clean)

In [28]:
def get_stemmed_text(corpus):
    stemmer = PorterStemmer()
    return [' '.join([stemmer.stem(word) for word in review.split()]) for review in corpus]
stemmed_reviews_train = get_stemmed_text(no_stop_words_train)
stemmed_reviews_test = get_stemmed_text(no_stop_words_valid)

In [29]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(stemmed_reviews_train)
X = tfidf_vectorizer.transform(stemmed_reviews_train)
X_test = tfidf_vectorizer.transform(stemmed_reviews_test)

In [30]:
X_train, X_val, y_train, y_val = train_test_split(X, target, train_size = 0.7)

In [31]:
text_classifier = RandomForestClassifier(n_estimators=10, random_state=0, verbose = 1)
text_classifier.fit(X_train, y_train)
y_pred=text_classifier.predict(X_val)
print(classification_report(y_val,y_pred))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   34.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s finished


              precision    recall  f1-score   support

  Irrelevant       0.90      0.82      0.86      3791
    Negative       0.89      0.90      0.89      6877
     Neutral       0.90      0.86      0.88      5421
    Positive       0.83      0.89      0.86      6110

    accuracy                           0.87     22199
   macro avg       0.88      0.87      0.87     22199
weighted avg       0.87      0.87      0.87     22199



In [34]:
print(X_val," ",y_pred)

  (0, 7422)	0.1930443673974154
  (0, 8269)	0.4696012678473787
  (0, 10125)	0.16840169464729332
  (0, 10482)	0.2043276634663615
  (0, 11196)	0.2993249242660739
  (0, 11486)	0.23183684192909754
  (0, 12895)	0.2012876951064786
  (0, 15742)	0.12477621591002425
  (0, 17977)	0.1906436645312167
  (0, 19714)	0.21981536937286927
  (0, 21292)	0.30913554234971674
  (0, 23215)	0.15550903852323983
  (0, 23274)	0.20043425012453225
  (0, 23522)	0.33585209559128143
  (0, 23838)	0.16033078712943968
  (0, 26164)	0.14109309490171337
  (0, 27990)	0.17845630820102437
  (0, 28199)	0.1944941679484147
  (1, 956)	0.41318231847280246
  (1, 3670)	0.27315738113617843
  (1, 3766)	0.2247351731468944
  (1, 9598)	0.36530375406416526
  (1, 10504)	0.25096451234364636
  (1, 11411)	0.2903730363242916
  (1, 13010)	0.29348145561931177
  :	:
  (22195, 25628)	0.2844594408520723
  (22196, 2448)	0.32514789947246486
  (22196, 4061)	0.35455674357071004
  (22196, 10747)	0.26461038653157676
  (22196, 15742)	0.28393556201506576
  (